# 第三章 高效的函数

## 3.1 函数是python的头等对象

In [1]:
def yell(text):
    return text.upper() + '!'

In [3]:
yell('hello')

'HELLO!'

### 3.1.1函数是对象

In [4]:
bark = yell

In [5]:
bark('woof')

'WOOF!'

In [6]:
del yell

In [7]:
yell('hello')

NameError: name 'yell' is not defined

In [8]:
bark('hey')

'HEY!'

In [9]:
bark.__name__

'yell'

指向函数的__变量与函数__本身是独立的

### 3.1.2 函数可存储在数据结构中

In [10]:
funcs = [bark,str.lower,str.capitalize]

In [11]:
funcs

[<function __main__.yell(text)>,
 <method 'lower' of 'str' objects>,
 <method 'capitalize' of 'str' objects>]

In [12]:
for f in funcs:
    print(f,f('hey there'))

<function yell at 0x00000230418AB488> HEY THERE!
<method 'lower' of 'str' objects> hey there
<method 'capitalize' of 'str' objects> Hey there


In [13]:
funcs[0]

<function __main__.yell(text)>

### 3.1.3 函数可传递给其他函数

In [14]:
def greet(func):
    greeting = func('Hi,I am a python program')
    return greeting

In [15]:
greet(bark)

'HI,I AM A PYTHON PROGRAM!'

In [16]:
def whisper(text):
    return text.lower()+'...'

In [17]:
greet(whisper)

'hi,i am a python program...'

能接受其他函数作为参数的函数称为__高阶__函数。

In [18]:
list(map(bark,['hello','hey','hi']))#map function

['HELLO!', 'HEY!', 'HI!']

### 3.1.4 函数可以嵌套

In [22]:
def speak(text):
    def whispers(t):
        return t.lower() + "..."
    return whisper(text)

In [23]:
speak('Hello')

'hello...'

但是whispers仅存在于speak 函数内部

In [24]:
whispers('Yo')

NameError: name 'whispers' is not defined

In [25]:
def get_speak_func(volume):
    def whisper(text):
        return text.lower() + '...'
    def yell(text):
        return text.upper() + '!'
    if volume > 0.5:
        return yell
    else:
        return whisper

In [26]:
get_speak_func(0.3)

<function __main__.get_speak_func.<locals>.whisper(text)>

In [27]:
get_speak_func(0.7)

<function __main__.get_speak_func.<locals>.yell(text)>

In [28]:
speak_func = get_speak_func(0.7)

In [29]:
speak_func('Hello')

'HELLO!'

### 3.1.5 函数可捕捉局部状态

内部函数不仅可以从父函数返回，还可以捕获并携带父函数的某些状态

In [30]:
def get_speak_func(text,volume):
    def whisper():
        return text.lower() + '...'
    def yell():
        return text.upper() + '!'
    if volume>0.5:
        return yell
    else:
        return whisper

In [33]:
get_speak_func('Hello,World',0.7)()

'HELLO,WORLD!'

In [34]:
def make_adder(n):
    def add(x):
        return x+n
    return add

In [35]:
plus_3 = make_adder(3)
plus_5 = make_adder(5)

In [36]:
plus_3(4)

7

In [37]:
plus_5(4)

9

In [38]:
make_adder(3)(4)

7

### 3.1.6 对象也可作为函数使用

In [39]:
class Adder:
    def __init__(self,n):
        self.n = n
    def __call__(self,x):
        return self.n + x

In [40]:
plus_3 = Adder(3)

In [41]:
plus_3(4)

7

在幕后，像函数那样“调用”一个对象实例实际上是在尝试执行该对象的__call__方法

内置的callable函数用于检查一个对象是否可以调用

In [42]:
callable(plus_3)

True

In [44]:
callable('hel;lo')

False

### 3.1.7 关键要点

python中一切皆为对象。函数也不例外。可以将函数分配给变量或存储在数据结构中作为头等对象，函数还可以被传递给其他函数或者作为其他函数的返回值。

头等函数的特性可以用来抽象并传递程序中的行为

函数可以嵌套，并且可以捕获并携带父函数的一些状态。具有这种行为的函数称为闭包

对象可以被设置为可调用的，因此很多情况下可以将其作为函数对待

## 3.2 lambda是单表达式函数

python中的lambda关键字可用来快速声明小型匿名函数

In [1]:
add = lambda x,y: x+y

In [2]:
add(5,3)

8

In [3]:
def add(x,y):
    return x+y

In [4]:
add(5,3)

8

In [5]:
(lambda x,y:x+y)(5,3)

8

### 3.2.1 lambda的使用场景

一般在对可迭代对象进行排序时，使用lambda表达式定义简短的key函数

In [6]:
tuples = [(1,'d'),(2,'b'),(3,'a'),(4,'c')]
sorted(tuples,key = lambda x:x[1])

[(3, 'a'), (2, 'b'), (4, 'c'), (1, 'd')]

In [7]:
sorted(range(-5,6), key = lambda x: x*x)

[0, -1, 1, -2, 2, -3, 3, -4, 4, -5, 5]

In [8]:
def make_adder(n):
    return lambda x: x+n

In [9]:
plus_3 = make_adder(3)
plus_5 = make_adder(5)

In [10]:
plus_3(4)

7

In [11]:
plus_5(4)

9

### 3.2.2 不应过度使用lambda函数

In [12]:
#有害
class Car:
    rev = lambda self : print('Wroom!')
    crash = lambda self:print('Boom!')

In [13]:
my_car = Car()

In [14]:
my_car.crash()

Boom!


In [15]:
#有害
list(filter(lambda x: x%2 == 0,range(16)))

[0, 2, 4, 6, 8, 10, 12, 14]

In [17]:
#清晰
[x for x in range(16) if x%2 == 0]

[0, 2, 4, 6, 8, 10, 12, 14]

### 3.2.3 关键要点

lambda函数是单表达式函数，不必与名称绑定（匿名）

lambda函数不能使用普通的pyth语句，其中总是包含着一个隐式的return语句

使用前总是先问问自己：使用普通具名函数或者解析式是否更清晰

## 3.3 装饰器的力量

对于理解装饰器来说，“头等函数”中最重要的特性有：

1.__函数是对象__，可以分配给变量并传递给其他函数，以及从其他函数返回

2.__在函数内部也能定义函数__,且子函数可以捕获父函数的局部状态（词法闭包）

### 3.3.1 python装饰器基础

In [18]:
def null_decorator(func):
    return func

In [19]:
def greet():
    return 'Hello!'

In [20]:
greet = null_decorator(greet)

In [21]:
greet()

'Hello!'

In [22]:
@null_decorator
def greet():
    return 'Hey'

In [23]:
greet()

'Hey'

### 3.3.2 装饰器可以修改行为

In [24]:
def uppercase(func):
    def wrapper():
        original_result = func()
        modified_result = original_result.upper()
        return modified_result
    return wrapper

In [25]:
@uppercase
def greet():
    return 'Hello!'

In [26]:
greet

<function __main__.uppercase.<locals>.wrapper()>

In [27]:
greet()

'HELLO!'

装饰器通过包装闭包来修改可调用对象的行为，因此无需永久性地修改原对象。原可调用对象的行为仅在装饰时才会改变。

### 3.3.3 将多个装饰器应用于一个函数

In [39]:
def strong(func):
    def wrapper():
        return '<strong>' + func() + '</strong>'
    return wrapper

In [40]:
def emphasis(func):
    def wrapper():
        return '<em>' + func() + '</em>'
    return wrapper

In [41]:
@strong
@emphasis

def greet():
    return 'Hello!'

In [42]:
greet()

'<strong><em>Hello!</em></strong>'

In [43]:
decorated_greet = strong(emphasis(greet))

### 3.3.4 装饰器接受参数的函数

In [44]:
def proxy(func):
    def wrapper(*args,**kwargs):
        return func(*args,**kwargs)
    return wrapper

In [45]:
def trace(func):
    def wrapper(*args,**kwargs):
        print(f'TRACE:calling {func.__name__}()'
             f'with {args},{kwargs}')
        original_result = func(*args,**kwargs)
        print(f'TRACE:{func.__name__}()'
             f'returned {original_result!r}')
        return original_result
    return wrapper

In [46]:
@trace
def say(name,line):
    return f'{name}:{line}'

In [47]:
say('Jane','Hello,World')

TRACE:calling say()with ('Jane', 'Hello,World'),{}
TRACE:say()returned 'Jane:Hello,World'


'Jane:Hello,World'

### 3.3.5 如何编写“可调试“的装饰器

In [49]:
def greet():
    """ Return a friendly greeting"""
    return 'Hello!'

In [50]:
@uppercase
def greet():
    """ Return a friendly greeting"""
    return 'Hello!'

In [51]:
greet.__name__

'wrapper'

In [54]:
print(greet.__doc__)

None


包装闭包隐藏了原函数的名称、文档字符、参数列表

In [55]:
import functools
def uppercase(func):
    @functools.wraps(func)
    def wrapper():
        return func().upper()
    return wrapper

In [56]:
@uppercase
def greet():
    """ Return a friendly greeting"""
    return 'Hello!'

In [57]:
greet.__name__

'greet'

In [58]:
greet.__doc__

' Return a friendly greeting'

### 3.3.6 关键要点

1。装饰器用于定义可重用的组件，可以将其应用于可调用对象以修改其行为，同时无需永久修改可调用对象本身。

2.@语法只是在输入函数上调用装饰器的简写。在单个函数上应用多个装饰器的顺序是从底部到顶部（装饰器栈）

3.为了方便调试，最好在自己的装饰器中使用functools.wraps将被装饰对象中的元数据转移到装饰后的对象中

## 3.4 有趣的*** args 和 **** kwargs

In [69]:
def foo(required,*args,**kwargs):
    print(required)
    if args:
        print(args)
    if kwargs:
        print(kwargs)

In [70]:
foo()

TypeError: foo() missing 1 required positional argument: 'required'

In [71]:
foo('hello')

hello


In [72]:
foo('hello',1,2,3)

hello
(1, 2, 3)


In [73]:
foo('hello',1,2,3,key1 = 'value',key2 = 999)

hello
(1, 2, 3)
{'key1': 'value', 'key2': 999}


### 3.4.1 传递可选参数或关键字参数

In [74]:
def foo(x,*args,**kwargs):
    kwargs['name'] = 'Alice'
    new_args = args + ('extra',)
    bar(x,*new_args,**kwargs)

In [75]:
class Car:
    def __init__(self,color,mileage):
        self.color = color
        self.mileage = mileage

class AlwaysBuleCar(Car):
    def __init__(self,*args,**kwargs):
        super().__init__(*args,**kwargs)
        self.color = 'bule'

In [76]:
AlwaysBuleCar('green',48392).color

'bule'

## 3.5 函数参数解包

In [77]:
def print_vector(x,y,z):
    print('<%s,%s,%s>' % (x,y,z))

In [78]:
print_vector(0,1,0)

<0,1,0>


In [79]:
tuple_vec = (1,0,1)
list_vec = [1,0,1]
print_vector(tuple_vec[0],
            tuple_vec[1],
            tuple_vec[2])

<1,0,1>


In [81]:
print_vector(*tuple_vec)

<1,0,1>


In [82]:
print_vector(*list_vec)

<1,0,1>


In [83]:
genexpr = (x*x for x in range(3))

In [84]:
print_vector(*genexpr)

<0,1,4>


In [85]:
dict_vec = {'y':0,'z':1,'x':1}

In [86]:
print_vector(**dict_vec)

<1,0,1>


In [87]:
print_vector(*dict_vec)

<y,z,x>


## 3.6 返回空值

python在所有函数的末尾隐式的添加了return None 语句

In [88]:
def foo1(value):
    if value:
        return value
    else:
        return None

In [89]:
def foo2(value):
    """纯return语句，相当于'return None'"""
    if value:
        return value
    else:
        return 

In [90]:
def foo3(value):
    """无return语句，也相当于'return None'"""
    if value:
        return value

In [92]:
type(foo1(0))

NoneType

In [93]:
type(foo2(0))

NoneType

In [94]:
type(foo3(0))

NoneType